In [1]:
import json
import pickle
from os import listdir

import pandas as pd
import spotipy
from spotipy import SpotifyClientCredentials
from tqdm.notebook import tqdm

В данном ноутбуке мы достаём из Спотифая фичи треков, спаршенных из EveryNoise (для всех доступных в EveryNoise жанров)

In [ ]:
# TODO Пофиксить импорты, т.к. пиклы переброшены в data/interim

In [2]:
PATH_TO_CFG = ""

with open(PATH_TO_CFG, "r") as f:
    SPOTIFY_CONFIG = json.load(f)
client_id = SPOTIFY_CONFIG["client_id"]
client_secret = SPOTIFY_CONFIG["client_secret"]

sp = spotipy.Spotify(
    auth_manager=SpotifyClientCredentials(
        client_id=client_id, client_secret=client_secret
    )
)

### Process genres


#### First: collect tracks from playlists

In [10]:
with open("data/filtered_genres_playlist_fin.json", "r") as f:
    genres = json.load(f)

In [11]:
def embed_to_url(embed_url):
    playlist_id = embed_url.split("playlist:")[1]
    return playlist_id  # f"https://open.spotify.com/{playlist_id}"

In [12]:
to_playlist_db = {}
to_tracks_db = {}

In [28]:
if "genres_tracker.pickle" in listdir():
    with open("genres_tracker.pickle", "rb") as f:
        genres_to_download = pickle.load(f)
else:
    genres_to_download = list(genres.keys())

genres_iter = genres_to_download.copy()
for genre_to_download in tqdm(genres_iter):
    # write playlists info
    playlist_id = embed_to_url(genres[genre_to_download])
    pl = sp.playlist(playlist_id)
    total_followers = pl["followers"]["total"]
    total_tracks = pl["tracks"]["total"]
    to_playlist_dict = {
        "playlist_id": playlist_id,
        "genre": genre_to_download,
        "total_followers": total_followers,
        "total_tracks": total_tracks,
    }

    # write tracks_info
    tracks_boxes = pl["tracks"]["items"]
    to_tracks_dict = {playlist_id: {}}
    for track_box in tracks_boxes:
        track = track_box["track"]
        if track is None:
            print("Found non-track")
            continue
        track_id = track["id"]
        to_tracks_dict[playlist_id].update({track_id: {}})

        # album info
        album_box = track["album"]
        to_tracks_dict[playlist_id][track_id].update(
            {
                "album_type": album_box["album_type"],
                "album_id": album_box["id"],
                "album_name": album_box["name"],
                "album_release_data": album_box["release_date"],
                "album_artists_names": ":artist_custom_separator:".join(
                    [i["name"] for i in album_box["artists"]]
                ),
                "album_artists_ids": ":artist_custom_separator:".join(
                    [i["id"] for i in album_box["artists"]]
                ),
                "album_total_tracks": album_box["total_tracks"],
            }
        )

        # artist info
        to_tracks_dict[playlist_id][track_id].update(
            {
                "track_artists_names": ":artist_custom_separator:".join(
                    [i["name"] for i in track["artists"]]
                ),
                "track_artists_ids": ":artist_custom_separator:".join(
                    [i["id"] for i in track["artists"]]
                ),
            }
        )

        # track info
        to_tracks_dict[playlist_id][track_id].update(
            {
                "track_name": track["name"],
                "track_id": track["id"],
                "track_type": track["type"],
                "track_duration_ms": track["duration_ms"],
                "track_explicit": track["explicit"],
                "track_popularity": track["popularity"],
            }
        )

    to_playlist_db.update({genre_to_download: to_playlist_dict})
    to_tracks_db.update(to_tracks_dict)
    genres_to_download.pop(0)
    with open("genres_tracker.pickle", "wb") as f:
        pickle.dump(genres_to_download, f)

    with open("to_playlist_db.pickle", "wb") as f:
        pickle.dump(to_playlist_db, f)

    with open("to_tracks_db.pickle", "wb") as f:
        pickle.dump(to_tracks_db, f)

  0%|          | 0/1430 [00:00<?, ?it/s]

Found non-track
Found non-track
Found non-track
Found non-track
Found non-track
Found non-track
Found non-track
Found non-track
Found non-track
Found non-track
Found non-track
Found non-track
Found non-track
Found non-track
Found non-track
Found non-track
Found non-track
Found non-track
Found non-track
Found non-track
Found non-track
Found non-track
Found non-track
Found non-track
Found non-track
Found non-track
Found non-track
Found non-track
Found non-track
Found non-track
Found non-track
Found non-track


#### Second: collect more track-level info

In [3]:
with open("to_tracks_db.pickle", "rb") as f:
    to_tracks_db = pickle.load(f)

In [4]:
if "to_tracks_db_second.pickle" in listdir():
    with open("to_tracks_db_second.pickle", "rb") as f:
        to_tracks_db_second = pickle.load(f)
else:
    to_tracks_db_second = {}

In [7]:
def split_into_chunks(iterable_, size):
    if size > len(iterable_):
        return None

    n_full = len(iterable_) // size
    to_return = []
    for i in range(n_full):
        to_return.append(iterable_[i * size : (i + 1) * size])
    if last_part := iterable_[n_full * size :]:
        to_return.append(last_part)
    return to_return

In [21]:
if "feature_chunks.pickle" in listdir():
    with open("feature_chunks.pickle", "rb") as f:
        chunks = pickle.load(f)
else:
    to_download = []
    for playlist_id in to_tracks_db:
        for track_id in to_tracks_db[playlist_id]:
            to_download.append([playlist_id, track_id])
    chunks = split_into_chunks(to_download, 100)

chunks_ready = chunks.copy()

for chunk in tqdm(chunks):
    playlist_ids = [i[0] for i in chunk]
    track_ids = [i[1] for i in chunk]
    features_data = sp.audio_features(track_ids)

    for feature_set in features_data:
        if feature_set is None:
            continue
        track_id = feature_set["id"]
        to_tracks_db_second.update({track_id: feature_set})
    chunks_ready.pop(0)

    with open("feature_chunks.pickle", "wb") as f:
        pickle.dump(chunks_ready, f)

    with open("to_tracks_db_second.pickle", "wb") as f:
        pickle.dump(to_tracks_db_second, f)

  0%|          | 0/319 [00:00<?, ?it/s]

### Unite files

In [23]:
with open("to_playlist_db.pickle", "rb") as f:
    to_playlist_db = pickle.load(f)

with open("to_tracks_db.pickle", "rb") as f:
    to_tracks_db = pickle.load(f)

In [24]:
to_df = []

for genre_name in to_playlist_db:
    playlist_id = to_playlist_db[genre_name]["playlist_id"]
    items = []
    for track_id in to_tracks_db_second:
        to_items = []
        try:
            to_items.extend(list(to_tracks_db[playlist_id][track_id].items()))
            to_items.extend(list(to_tracks_db_second[track_id].items()))
            to_items.extend(list(to_playlist_db[genre_name].items()))
        except Exception:
            continue
        items.append(to_items)
    for item in items:
        to_df.append({key: val for key, val in item})

In [25]:
df = pd.DataFrame(to_df)

In [26]:
df

,album_type,album_id,album_name,album_release_data,album_artists_names,album_artists_ids,album_total_tracks,track_artists_names,track_artists_ids,track_name,...,id,uri,track_href,analysis_url,duration_ms,time_signature,playlist_id,genre,total_followers,total_tracks
0,album,6Kssm2LosQ0WyLukFZkEG5,Demi,2013-01-01,Demi Lovato,6S2OmqARrzebs0tKUEyXyp,13,Demi Lovato,6S2OmqARrzebs0tKUEyXyp,Heart Attack,...,1V6gIisPpYqgFeWbMLI0bA,spotify:track:1V6gIisPpYqgFeWbMLI0bA,https://api.spotify.com/v1/tracks/1V6gIisPpYqg...,https://api.spotify.com/v1/audio-analysis/1V6g...,210840,4,6gS3HhOiI17QNojjPuPzqc,pop,22857.0,428
1,single,65L5VcKGKe6sOJIPNFc0Q1,One in a Million,2023-08-04,Bebe Rexha:artist_custom_separator:David Guetta,64M6ah0SkkRsnPGtGiRAbb:artist_custom_separator...,1,Bebe Rexha:artist_custom_separator:David Guetta,64M6ah0SkkRsnPGtGiRAbb:artist_custom_separator...,One in a Million,...,3YfGTvsTAWGC2lgoDOikUz,spotify:track:3YfGTvsTAWGC2lgoDOikUz,https://api.spotify.com/v1/tracks/3YfGTvsTAWGC...,https://api.spotify.com/v1/audio-analysis/3YfG...,160530,4,6gS3HhOiI17QNojjPuPzqc,pop,22857.0,428
2,single,5Q5PomrTdI13qpmEeM393B,On My Love (Sped Up),2023-09-12,Zara Larsson:artist_custom_separator:David Guetta,1Xylc3o4UrD53lo9CvFvVg:artist_custom_separator...,4,Zara Larsson:artist_custom_separator:David Guetta,1Xylc3o4UrD53lo9CvFvVg:artist_custom_separator...,On My Love,...,0K6iKgHPPhAb4Acmg9aD77,spotify:track:0K6iKgHPPhAb4Acmg9aD77,https://api.spotify.com/v1/tracks/0K6iKgHPPhAb...,https://api.spotify.com/v1/audio-analysis/0K6i...,222912,4,6gS3HhOiI17QNojjPuPzqc,pop,22857.0,428
3,album,6fpLLJsDSSAlToEDW2jv4F,Animal (Expanded Edition),2010-01-01,Kesha,6LqNN22kT3074XbTVUrhzX,18,Kesha,6LqNN22kT3074XbTVUrhzX,TiK ToK,...,0HPD5WQqrq7wPWR7P7Dw1i,spotify:track:0HPD5WQqrq7wPWR7P7Dw1i,https://api.spotify.com/v1/tracks/0HPD5WQqrq7w...,https://api.spotify.com/v1/audio-analysis/0HPD...,199693,4,6gS3HhOiI17QNojjPuPzqc,pop,22857.0,428
4,album,20Ol6zZ0nLlc5EGTH1zA0j,Delirium (Deluxe),2015-11-06,Ellie Goulding,0X2BH1fck6amBIoJhDVmmJ,22,Ellie Goulding,0X2BH1fck6amBIoJhDVmmJ,"Love Me Like You Do - From ""Fifty Shades Of Grey""",...,3zHq9ouUJQFQRf3cm1rRLu,spotify:track:3zHq9ouUJQFQRf3cm1rRLu,https://api.spotify.com/v1/tracks/3zHq9ouUJQFQ...,https://api.spotify.com/v1/audio-analysis/3zHq...,252534,4,6gS3HhOiI17QNojjPuPzqc,pop,22857.0,428
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145727,album,40MtmStfAvh7Vs08hZhA4X,Tae The Green Woods Gaen,2001-01-01,Jock Duncan,4HYx5HNAsAtAN6StGZHnLl,17,Jock Duncan,4HYx5HNAsAtAN6StGZHnLl,Sir Patrick Spens,...,1RzTdUBG7HFiq8Z6TrdEEB,spotify:track:1RzTdUBG7HFiq8Z6TrdEEB,https://api.spotify.com/v1/tracks/1RzTdUBG7HFi...,https://api.spotify.com/v1/audio-analysis/1RzT...,268613,3,4VIb2spYu2TVsYw9FmWne6,bothy ballad,34.0,65
145728,compilation,1mcAaCph2TjInqdDfF9sPJ,The Bothy Songs and Ballads of North East Scot...,2001-01-20,Various Artists,0LyfQWJT6nXafLPZqxe9Of,15,Jimmy McBeath,1SLrenZTCwlh16dBTjRHup,Gruel,...,0Tr3gJoBSwLRy5VCw1Eies,spotify:track:0Tr3gJoBSwLRy5VCw1Eies,https://api.spotify.com/v1/tracks/0Tr3gJoBSwLR...,https://api.spotify.com/v1/audio-analysis/0Tr3...,137933,4,4VIb2spYu2TVsYw9FmWne6,bothy ballad,34.0,65
145729,album,6RgkCjTOnXvMGU1tGRUgyN,Behind The Bothy Door Volume 1,1988-01-01,Tam Reid,4MFEL2IVUuZHUONDl7jqhC,15,Tam Reid,4MFEL2IVUuZHUONDl7jqhC,Princie And Jean,...,7BDGc2Mrn8koWnkLHgxdCQ,spotify:track:7BDGc2Mrn8koWnkLHgxdCQ,https://api.spotify.com/v1/tracks/7BDGc2Mrn8ko...,https://api.spotify.com/v1/audio-analysis/7BDG...,208440,4,4VIb2spYu2TVsYw9FmWne6,bothy ballad,34.0,65
145730,album,4y79n4PgbbCoHfd50NwFwE,Songs from the Irish Tradition,1982,Mary-Ann Carolan,3FbtkRtF5kVsxIffoyQmyV,10,Mary-Ann Carolan,3FbtkRtF5kVsxIffoyQmyV,The Maid of Ballymore,...,4w5Ce2oVu3cUbEDDtjOc6w,spotify:track:4w5Ce2oVu3cUbEDDtjOc6w,https://api.spotify.com/v1/tracks/4w5Ce2oVu3cU...,https://api.spotify.com/v1/audio-analysis/4w5C...,233360,3,4VIb2spYu2TVsYw9FmWne6,bothy ballad,34.0,65


In [27]:
with open("../data/small_genres_tracks_df.pickle", "wb") as f:
    pickle.dump(df, f)